# デレステのwikiページからアイドルコメントの情報を抽出する
まずアイドル名とそれに紐づいたurlのリストを生成する。

In [2]:
import requests, bs4, re
#各属性のアイドルが一覧に示されているページ
type_temp = "passion"
if type_temp == "cute":
    idole_list = "https://seesaawiki.jp/imascg/d/%a5%a2%a5%a4%a5%c9%a5%eb%b0%ec%cd%f7%a1%ca%a5%ad%a5%e5%a1%bc%a5%c8%a1%cb"
if type_temp == "cool":
    idole_list = "https://seesaawiki.jp/imascg/d/%a5%a2%a5%a4%a5%c9%a5%eb%b0%ec%cd%f7%a1%ca%a5%af%a1%bc%a5%eb%a1%cb"
if type_temp == "passion":
    idole_list = "https://seesaawiki.jp/imascg/d/%a5%a2%a5%a4%a5%c9%a5%eb%b0%ec%cd%f7%a1%ca%a5%d1%a5%c3%a5%b7%a5%e7%a5%f3%a1%cb"

res = requests.get(idole_list)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, "html.parser")
elems = soup.select('td')#とりあえずtdを取り出す
number = 0
url_list = []
name_list = []
a1 = "\""
b1 = "\""
a2 = "\">"
b2 = "</a>"
for elem in elems:#tdの中からさらにurlと名前を抽出
    if "<td><a" in str(elem):
        string = re.findall(a1+".*"+b1,str(elem))[0]#re.findallではリスト形式にされるのでstrとして抽出
        url_list.append(string[1:len(string)-1])#端に""がつくのでそれを削除
        string = re.findall(a2+".*"+b2,str(elem))[0]
        name_list.append(string[2:len(string)-4])
if type_temp == "cute":
    error_list = [396]#杏のデータに不備?
if type_temp == "cool":
    error_list = [394]#ライラのデータに不備?
if type_temp == "passion":
    error_list = []

url_list = [url_list[i] for i in range(len(url_list)) if i not in error_list]
name_list = [name_list[i] for i in range(len(name_list)) if i not in error_list]
print("name number:",len(name_list))
print("url number:",len(url_list))
for i in range(10):#最初の方だけ表示する
    print(name_list[i])
    print(url_list[i])

name number: 1046
url number: 1046
本田未央
https://seesaawiki.jp/imascg/d/%cb%dc%c5%c4%cc%a4%b1%fb
本田未央+
https://seesaawiki.jp/imascg/d/%cb%dc%c5%c4%cc%a4%b1%fb#+
高森藍子
https://seesaawiki.jp/imascg/d/%b9%e2%bf%b9%cd%f5%bb%d2
高森藍子+
https://seesaawiki.jp/imascg/d/%b9%e2%bf%b9%cd%f5%bb%d2#+
並木芽衣子
https://seesaawiki.jp/imascg/d/%ca%c2%cc%da%b2%ea%b0%e1%bb%d2
並木芽衣子+
https://seesaawiki.jp/imascg/d/%ca%c2%cc%da%b2%ea%b0%e1%bb%d2#+
龍崎薫
https://seesaawiki.jp/imascg/d/%ce%b6%ba%ea%b7%b0
龍崎薫+
https://seesaawiki.jp/imascg/d/%ce%b6%ba%ea%b7%b0#+
木村夏樹
https://seesaawiki.jp/imascg/d/%cc%da%c2%bc%b2%c6%bc%f9
木村夏樹+
https://seesaawiki.jp/imascg/d/%cc%da%c2%bc%b2%c6%bc%f9#+


それぞれのurlに飛んでアイドルコメントを抽出する。一つのページにはほとんどの場合特訓前と特訓後のアイドルコメントが両方載っているので、うまいこと処理する必要がある。

In [4]:
comment = [[] for i in range(len(name_list))]
i  = 0
print("total: ",len(name_list)," times")
judge = 0
for url in url_list:
    print(i,"times",end='\r')
    if judge == 1:#judgeはひとつ前のurlに特訓前後両方のアイドルコメントがあったかどうかを判定する
        comment[i] = string#両方のアイドルコメントがあったとき、事前にstringに格納しておいた特訓後のアイドルコメントを入れる
        judge = 0
    else:
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        elems_idle = soup.select('td')#とりあえずtdを取り出す
        counter = 0
        flag = 0#flagはelemにアイドルコメントという文字列が含まれているかを判定する、含まれていれば次にアイドルコメントがくる
        for elem in elems_idle:#tdを含むものからアイドルコメントを抽出する
            if flag == 1 and counter == 1:#counterはアイドルコメントが載っている数を示す、2つあった場合は特訓後のコメントが載っていることになる
                comment[i] = str(elem)[4:len(str(elem))-5]
                flag = 0
            if flag == 1 and counter == 2:
                string = str(elem)[4:len(str(elem))-5]
                flag = 0
                judge = 1
            if "アイドルコメント" in str(elem):
                flag = 1
                counter += 1
    #print(comment[i])
    i += 1
if type_temp == "cute":#"ベージュ"という文字列のせいでうまくいかないため例外処理
    comment[1006] = "あっ、○○プロデューサー、おはようございますっ！ へへっ、これから事務所に行くところだったんです。一緒に行きましょうか！ …えっ？ ボクの制服姿、気になりますか？"
    comment[1007] = "あっ、○○プロデューサー、おはようございますっ！ へへっ、これから事務所に行くところだったんです。一緒に行きましょうか！ …コレですか？また女の子からラブレターをもらっちゃって！"
    comment[1008] = "あっ、○○プロデューサー、おはようございますっ。これから事務所に行くところだったんです。一緒にいきましょうか…えっ？ いやぁ、女の子からのラブレターが…その…あは、あははは！"
    comment[1009] = "あ…○○プロデューサー、おはよう…ございます…これから事務所に行くところ…です。一緒に行きましょうか…え？ あぁ…その、女子からのラブレターが…イッパイ…アハハハハ…ハハ…"
if type_temp == "cool":#脚注があるなどの理由で例外処理
    comment[211] = "カーク ヴァス ザヴート？ ………フッ、貴方のお名前は？ と聞きました。お名前は？ …○○、ね。…ミーニャ ザヴート アーニャ。<br/>アーニャは、ええと…ニックネームよ。私はアーニャ…アナスタシアです。よろしく、プロデューサー。"
    comment[212] = "ンー…ニチェボー…悪くないです。アイドルは皆のナジェージタだから…あー…希望ですから、こういうドレスも素敵ですね。<br/>…これからアイドル頑張りますから、○○プロデューサー、お願いしますね"
    comment[213] = "ハァイ、まさかアイドルのお仕事でhomeに帰ってくるなんて思ってなかったデス。ペアサイダーも懐かしいネ！ 日本じゃあまり売ってなくて…。プロデューサーの分も買ってきたから一緒に飲みましょ？"
    comment[249] = "○○プロデューサー、ズヴェズダ、好きですか？…あー、星です。この空の上にはコースモス、宇宙、ですね。<br/>知らない星がたくさんある。ワクワクしますね。アイドルも同じです。知らないことが、たくさん"
    comment[250] = "○○プロデューサー、この服はコスモナーフトですか？あー…ええと、宇宙…飛行士？ですか？とてもすてきですね。<br/>アイドルはとても難しい、ですけど、楽しいからアイドルアーニャ、旅立ちますね"
    comment[1014] = "わ、私…旅行とか…い、行ったこと無い、です…。外、出ないし…遠くは行かないから…。だから…プロデューサーさん、じ、準備とか…て、手伝ってくれ…たら…う、うう、嬉しい…です…"
    comment[1015] = "わ、私…旅行とか…い、行ったこと無い、です…。外、出ないし…遠くは行かないから…。だから…プロデューサーさん、じ、準備とか…て、手伝ってくれ…たら…う、うう、嬉しい…です…"
    comment[1016] = "わ、私…旅行とか…い、行ったこと無い、です…。外、出ないし…遠くは行かないから…。だから…プロデューサーさん、じ、準備とか…て、手伝ってくれ…たら…う、うう、嬉しい…です…"
    comment[1017] = "わ、私…旅行とか…い、行ったこと無い、です…。外、出ないし…遠くは行かないから…。だから…プロデューサーさん、じ、準備とか…て、手伝ってくれ…たら…う、うう、嬉しい…です…"
    comment[1018] = "わ、私…旅行とか…い、行ったこと無い、です…。外、出ないし…遠くは行かないから…。だから…プロデューサーさん、じ、準備とか…て、手伝ってくれ…たら…う、うう、嬉しい…です…"
if type_temp == "passion":
    comment[402] = "カラフルなドレス…今の私の気持ちを現してるみたいね。ドキドキもワクワクもトキメキも…色んな想いが私の胸の中を駆け巡ってるわ。私が○○くんの色に染められちゃったみたいね。ふふっ♪"
    
N = len(comment)
string = comment[0]
i_current = 1
for i in range(len(comment)):
    comment[i] = comment[i].replace("<br/>","")#改行文字を削除
for i in range(1,N):#重複しているものの削除
    if comment[i_current] in comment[0:i_current]:
        comment.pop(i_current)
        name_list.pop(i_current)
    else:
        i_current += 1
for i in range(len(comment)):
    print(i," name:"+name_list[i],":",comment[i])


total:  1046  times
0  name:本田未央 : 本田未央15歳。高校一年生ですっ！ 元気に明るく、トップアイドル目指して頑張りまーっす！ えへへ。今日からよろしくお願いしまーす♪
1  name:本田未央+ : わ〜っ♪こんな可愛い衣装、ありがとうプロデューサー！！ 私、何だかやれそうな気がしてきたよー！ えへへ、私、これまで以上にいーっぱい頑張るから、これからもよろしくね！
2  name:高森藍子 : はじめまして、プロデューサーさん、ファンのみなさんが優しい気持ちになれるような、微笑んでくれるようなアイドルを目指したいと思っているんです。よろしくお願いしますねっ。
3  name:高森藍子+ : ファンのみなさんの笑顔、一人一人ちゃんと覚えるようにしてるんです。つらくてもそれを思い出せば、いつでも幸せな気持ちになれますからねっ。幸せな気持ち、大事ですっ。
4  name:並木芽衣子 : アイドルになったらどんなことが出来るかな？ 世界を飛び回ったりできたらすっごく楽しそうだよねっ！ よろしく、プロデューサーさん♪
5  name:並木芽衣子+ : まだまだ私の知らない世界ってあるんだね、もっといろんなものを見てみたいなっ！ ねぇプロデューサーさん、次はどんな世界を見つけられるのかなっ？
6  name:龍崎薫 : やっほー！ かおるだよーっ！！ ねぇねぇねぇ！！ アイドルってほんとになれる？ かおるはテレビに出てたことあるから、プロデューサーさんが先生になってくれたらだいじょぶかな！
7  name:龍崎薫+ : せんせぇ！ かおるね、みんながかわいいって言ってくれたんだけど、これってせんせぇのおかげだよね！ だから、もっとかわいいアイドルになりたいなって思うの！」
8  name:木村夏樹 : よっ！ アンタがプロデューサーさん？ アタシ、夏樹。ロックが好きなんだけど、アイドルも結構好きなんだよね。もしかしてロックアイドルってカッコいいし、可愛い感じで最強なんじゃない？ 目指してみっか！
9  name:木村夏樹+ : よぉプロデューサー！ アタシもアンタのおかげで立派なアイドルだし、心はロックに燃えてるんだよね！ でもアタシがなりたいのはスターだから、まだまだ止まらないで行くぜっ！
10  name:松山久美子 : 私ね、もっとキレイ

514  name:［おませなリトルガール］メアリー・コクラン : ショッピングに付き合ってくれてアリガト。おかげでいーっぱいカワイイお洋服が選べたワ。ほらこれなんてドウ？キュートなアタシにピッタリだと思わない？ダーリンのためにかわいくセクシーになりたいモン！
515  name:［おませなリトルガール］メアリー・コクラン+ : アタシほどカワイイアイドルはいないでしょ？いつもと違う衣装だってキュートにセクシーに着こなしちゃうんだからネ！ でも…アタシの魅力を1番分かってるのは〇〇かもしれないわネ♪
516  name:［シークレット★ハート］城ヶ崎美嘉 : ふーっ、プロデューサー、わざわざ送ってくれてありがとー★まぁまぁ、せっかく来てくれたんだしお茶でも飲んでけば！いいからいいから★アタシの部屋に上がれるなんてめったにないんだし！
517  name:［シークレット★ハート］城ヶ崎美嘉+ : みんなー、またねーっ★……あっ、プロデューサー、おっつー♪アツいLIVEだったでしょ？熱気、すごかったもんね♪ふぅ……でもステージから離れたらちょっと肌寒いね…あっ、かけてくれるの？優しー★
518  name:［CDデビュー］十時愛梨 : プロデューサーさん、私もついにCDデビューですっ！ 今日は○○さんへの感謝の気持ちを込めてケーキを作って来ちゃいました！ 愛梨特製ケーキ、いーっぱい召し上がれ♪
519  name:［CDデビュー］十時愛梨+ : この衣装、白くてふわふわで、何だか気分までふわふわ〜ってなりますね。えへへっ！自分のCDが出せて、こんな可愛い衣装まで着られて…私、幸せすぎてふわふわ〜って…えっ、もう出番！？
520  name:［CDデビュー］本田未央 : 私がCDデビューなんて、プロデューサーが冗談言ってるのかなって思ってた♪なーんて、ウソウソ！ アイドル本田未央、全力で頑張らせていただきますっ！ じゃ、二人三脚でいってみよーっ！ ごーごー♪
521  name:［CDデビュー］本田未央+ : じゃーん！ どうどう？ プロデューサー。似合ってる？ みんなとお揃いの衣装って嬉しいな〜。今日は、思いっきり楽しんじゃおうっ♪あ、でもでも、やりすぎだったら教えてね？ さー、楽しむぞー♪
522  name:［ビターバレンタイン］衛藤美紗希 : 今日はぁ、あたしのわ

889  name:［炎天の女王］向井拓海+ : なッ…！し、しまったッ！小せぇ相手だからって…油断しちまったッ！くッ…！まさかこのアタシが…勝負事で負けるなんてッ…！あァッ…ちょ、ちょっと待てッ！じ、実はアタシ、泳げな…ナァァァ！
890  name:［シューティングチアー］愛野渚 : よォし！一気に、いっちゃおうよォー！…………お、思ったより高いんだね…わ、わかってるって！後がつかえてるんでしょ？今、いくからッ！こ、こらッ！押さないでよッ！○○プロデューサー！
891  name:［シューティングチアー］愛野渚+ : 見よッ！この華麗なるモップさばきッ！プールサイドに、冷たく舞い散る水しぶき、みんなの火照った体をクールダウンッ！マリンボーダーの水兵さんが、アナタの心をガシガシ磨いてピッカピカにするからね
892  name:［ふんわり花乙女］相葉夕美 : 小さいころ、お花畑で花冠を作ったり、お花の指輪を作ったよねっ。今日はそんな気分でピクニックだよ♪そうだ、○○さんも、胸元にお花を挿してみる？気分が明るくなるかもっ♪
893  name:［ふんわり花乙女］相葉夕美+ : 今日はお花のバスケットに乗って、ファンのみんなのもとへいくからっ！みんなの笑顔をいっぱい咲かせてみせるから、楽しみにしていてね！それじゃあ、夕美のフラワーLIVE、スタートだよ
894  name:［ファントムレディ］的場梨沙 : 『狙った獲物は逃がさない、神出鬼没のセクシー怪盗！その名も…怪盗リサ！』フフッ、アタシにピッタリの役ね♪○○、アンタの期待通り、ヒロインの座は華麗に奪ってみせるわ。この財宝の山みたいにね！
895  name:［ファントムレディ］的場梨沙+ : 『アハハ、残念！そっちはニセモノよっ♪』…しなやかなアクション！鮮やかなトリック！どう？この華麗なる盗みの手口！ピカピカのダイヤと一緒に、みんなのハートも奪い去ってあげるわ♪覚悟しなさいっ！
896  name:［放課後サマー］高森藍子 : ○○さん、夏の日差しが…眩しいですね。この日差しの眩しさで、私の水着姿も隠されてしまったらいいのに…なんて。ここまできたら、もう、逃げられませんね。覚悟、決めますっ
897  name:［放課後サマー］高森藍子+ : みなさーん！今日は私たちが歌って踊って、みなさんに忘れられない思い出、作

毎回これを処理するのは面倒なのでテキストデータにまとめてしまう

In [5]:
with open("name_"+type_temp+".txt",mode = "w") as f:
    for string in name_list:
        f.write(string+"\n")
with open("comment_"+type_temp+".txt",mode = "w") as f:
    for string in comment:
        f.write(string+"\n")